In [1]:
# import required packages
import mealpy
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score
from globals.pandas_functions import *
from mealpy.swarm_based import PSO
from mealpy.swarm_based import GWO
from mealpy.swarm_based import FA
import globals.hyperparmeter_optimizer as hyp_optimizer
import globals.model_evaluations as evaluations
from mealpy.utils.space import IntegerVar, FloatVar, BoolVar, CategoricalVar
import globals.gpu_processing as gpu_processing
from tensorflow.keras import mixed_precision

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
# import datasets
# load preprocessed training and testing data
data_base_path = "../data/processed/null_value_option_1/scaled_and_balanced"

X_train = pd.read_csv(f"{data_base_path}/pca_selected_features/unified_transaction_data_option1_x_train_pca.csv")
X_test = pd.read_csv(f"{data_base_path}/pca_selected_features/unified_transaction_data_option1_x_test_pca.csv")
y_train = pd.read_csv(f"{data_base_path}/unified_transaction_data_option1_y_train_balanced.csv")
y_test = pd.read_csv(f"{data_base_path}/unified_transaction_data_option1_y_test.csv")

In [3]:
dataset_dimension("X_train", X_train)
dataset_dimension("X_test", X_test)

X_train dataset dimension: (911764, 11)
X_test dataset dimension: (118102, 11)


In [5]:
# above TF 2.10 no GPU native support

Num GPUs Available:  0


In [6]:
# configure TF for GPU processing
gpu_processing.configure_gpu(True, True, True)
tf_strategy = gpu_processing.get_tf_strategy(False)
tf_strategy = tf_strategy or tf.distribute.get_strategy()

GPUs: []


## MLP Model Implementation with Hyperparameter Optimization

In [7]:
# define optimizer properties
param_optimizer_algorithm = "PSO"   # "PSO", "GWO", "FA" (Firefly)
population = 12     # number of candidate solutions
iterations = 12     # optimization iterations (epochs in mealpy terms)
cross_validation_folds = 3        # stratified folds for the objective
epochs_for_evaluation = 12
batch_size = 1024
early_stopping = 3
seed = 42

In [9]:
# building ANN model
def build_mlp_model(hyperparameters, input_shape,  lr=0.001):

    with tf_strategy.scope():
        try:
            policy = mixed_precision.global_policy()
            mixed_active = (policy and policy.name == 'mixed_float16')
        except Exception:
            mixed_active = False

    # building the model

    inputs = tf.keras.Input(shape=(input_shape,), name="inputs")
    x = inputs
    for i, u in enumerate(hyperparameters["units_per_layer"]):
        x = tf.keras.layers.Dense(u, name=f"dense_{i+1}")(x)

        if hyperparameters["batch_norm"]:
           x = tf.keras.layers.BatchNormalization(name=f"bn_{i+1}")(x)

        x = hyp_optimizer.get_activation_layer(hyperparameters["activation"])(x)

        if hyperparameters["dropout_rate"] > 0:
            x = tf.keras.layers.Dropout(rate=hyperparameters["dropout_rate"], name=f"dropout_{i+1}")(x)

    out_dtype = "float32" if mixed_active else None  # IMPORTANT for mixed precision: keep the output in float32 for stable sigmoid/BCE
    outputs = tf.keras.layers.Dense(1, name="probability", activation="sigmoid", dtype=out_dtype)(x)
    mlp_model = tf.keras.Model(inputs, outputs, name="fraud_detection_mlp_model")
    mlp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="binary_crossentropy")

    return mlp_model

In [10]:
def set_optimizer_objective(X, y, cv,  max_epochs, batch_size, seed, early_stopping_patience):
    """
    Build the objective: minimize (1 - mean F1 across CV folds).
    Simplicity choices:
      - Fixed threshold=0.5 for F1
      - EarlyStopping on val_loss
      - No SMOTE here (assume your training set is already prepared)
    """
    splits = list(StratifiedKFold(n_splits=cv, shuffle=True, random_state=seed).split(np.zeros_like(y), y))

    def obj(vec):
        hp = hyp_optimizer.optimizer_vectors_to_mlp_hyperparams( vec)
        f1_scores = []
        for tr_idx, va_idx in splits:
            X_tr, y_tr = X[tr_idx], y[tr_idx]
            X_va, y_va = X[va_idx], y[va_idx]

            mlp_model = build_mlp_model(hp, X.shape[1], 1e-3)  # LR 0.001 fixed for simplicity
            es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=early_stopping_patience, restore_best_weights=True, verbose=0)
            mlp_model.fit(X_tr, y_tr, validation_data=(X_va, y_va),
                      epochs=max_epochs, batch_size=batch_size, verbose=0, callbacks=[es])

            y_prob = mlp_model.predict(X_va, verbose=0).ravel()
            f1_scores.append(evaluations.classification_metrics(y_va, y_prob, threshold=0.5)["f1"])

        # minimize 1 - mean(F1)
        return 1.0 - float(np.mean(f1_scores))

    return obj

In [11]:
# hyperparameter optimizer runner
def run_hyperparam_optimization(algorithm,
                  objective,
                  iterations=iterations,
                  population=population):
    """
    Run the selected mealpy optimizer and print per-epoch global best parameters.

    Parameters
    ----------
    algorithm : str
        One of {'PSO', 'GWO', 'FA'}.
    objective : callable
        Function f(vec) -> scalar (1 - mean F1). Created by make_objective().
    iterations : int
        Number of optimization epochs (mealpy 'epoch').
    population : int
        Population size (mealpy 'pop_size').

    Returns
    -------
    best_vec : array-like
        Final best solution vector (continuous).
    best_obj : float
        Final best objective value (1 - mean F1).
    best_hp : dict
        Decoded hyperparameters from best_vec.
    epoch_logs : list[dict]
        Per-epoch records with keys:
          'epoch', 'f1', 'objective', 'vector',
          'n_layers', 'units', 'activation', 'batch_norm', 'dropout'
    """
    # Bounds match vec_to_hp layout:
    # [n_layers, units1, units2, units3, units4, dropout, activation_code, batch_norm_flag, (unused), (unused)]
    lower_bounds = [1, 16, 16, 16, 16, 0.0, 0, 0, 0.0, 0.0]
    upper_bounds = [4, 256, 256, 256, 256, 0.5, 2, 1, 1.0, 1.0]

    # problem = dict(fit_func=objective, lb=lower_bounds, ub=upper_bounds, minmax="min", log_to=None)

    # problem_lbub = dict(fit_func=objective, lb=lower_bounds, ub=upper_bounds, minmax="min", log_to=None)
    # problem_bounds = dict(obj_func=objective, bounds=list(zip(lower_bounds, upper_bounds)), minmax="min", log_to=None)

    # Multiples of 8 from 16 to 256 inclusive


    optimizer_bounds = [
        IntegerVar(lb=1, ub=4),               # n_layers
        CategoricalVar(hyp_optimizer.hidden_layer_unit_choices),         # units1
        CategoricalVar(hyp_optimizer.hidden_layer_unit_choices),         # units2
        CategoricalVar(hyp_optimizer.hidden_layer_unit_choices),         # units3
        CategoricalVar(hyp_optimizer.hidden_layer_unit_choices),         # units4
        FloatVar(lb=0.0, ub=0.5),             # dropout
        CategoricalVar(hyp_optimizer.activation_functions),          # activation
        BoolVar(),                            # batch_norm
    ]

    problem = dict(obj_func=objective, bounds=optimizer_bounds, minmax="min", log_to=None)


    # Select optimizer
    if algorithm.upper() == "PSO":
        optimizer = PSO.OriginalPSO(epoch=iterations, pop_size=population)
    elif algorithm.upper() == "GWO":
        optimizer = GWO.OriginalGWO(epoch=iterations, pop_size=population)
    elif algorithm.upper() == "FA":
        optimizer = FA.OriginalFA(epoch=iterations, pop_size=population)
    else:
        raise ValueError("algorithm must be one of {'PSO','GWO','FA'}")

    # Run optimization
    best_vec, best_obj = optimizer.solve(problem)
    best_hp = hyp_optimizer.optimizer_vectors_to_mlp_hyperparams(best_vec)

    # Collect per-epoch history (mealpy stores global best progression)
    epoch_logs = []
    try:
        history_vectors = getattr(optimizer.history, "list_global_best", [])
        history_scores = getattr(optimizer.history, "list_global_best_fitness", [])
    except AttributeError:
        history_vectors, history_scores = [], []

    total_epochs = len(history_scores)

    print("\n=== Per-Epoch Global Best Log ===")

    for i in range(total_epochs):
        vec_epoch = history_vectors[i]
        obj_epoch = history_scores[i]
        f1_epoch = 1.0 - obj_epoch  # Because objective = 1 - F1
        hp_epoch = hyp_optimizer.optimizer_vectors_to_mlp_hyperparams(vec_epoch)
        log_record = {
            "epoch": i + 1,
            "f1": float(f1_epoch),
            "objective": float(obj_epoch),
            "vector": [float(v) for v in vec_epoch],
            "n_layers": hp_epoch["n_layers"],
            "units": hp_epoch["units"],
            "activation": hp_epoch["activation"],
            "batch_norm": hp_epoch["batch_norm"],
            "dropout": hp_epoch["dropout"],
        }
        epoch_logs.append(log_record)

        print(f"Epoch {i+1:02d}/{total_epochs} | F1={f1_epoch:.4f} | Layers={hp_epoch['n_layers']} "
              f"| Units={hp_epoch['units']} | Act={hp_epoch['activation']} "
              f"| BN={hp_epoch['batch_norm']} | Dropout={hp_epoch['dropout']:.3f}")

    print("\n=== Final Best Solution ===")
    print(f"Best objective (1 - F1): {best_obj:.6f}  => F1={1 - best_obj:.6f}")
    print("Best hyperparameters:", best_hp)

    return best_vec, best_obj, best_hp, epoch_logs

In [12]:
def retrain_and_evaluate(best_hp, X_train, y_train, X_test, y_test,
                         max_epochs=40, batch=batch_size, early_stopping_patience=8):
    """
    Retrain the best architecture on all training data (with val_split=0.1),
    then evaluate on the untouched test set at threshold=0.5.
    """
    mlp_model = build_mlp_model(best_hp, X_train.shape[1], 1e-3)
    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=early_stopping_patience, restore_best_weights=True)
    mlp_model.fit(X_train, y_train, validation_split=0.1, epochs=max_epochs, batch_size=batch, verbose=1, callbacks=[es])

    y_prob = mlp_model.predict(X_test, verbose=0).ravel()
    metrics = evaluations.classification_metrics(y_test, y_prob, threshold=0.5)
    return mlp_model, metrics

In [ ]:
# execute hyperparameter optimization
optimizer_algorithm = "PSO"   # "PSO", "GWO", "FA" (Firefly)
optimizer_objective = set_optimizer_objective(X_train.to_numpy(),
                                              y_train.to_numpy().ravel(),
                                              cv=cross_validation_folds,
                                              max_epochs=epochs_for_evaluation,
                                              batch_size=batch_size,
                                              seed=seed,
                                              early_stopping_patience=early_stopping)

print(f"\nRunning optimizer: {optimizer_algorithm} (pop={population}, iters={iterations})")

best_vector, best_objective, best_hyper_params = run_hyperparam_optimization(optimizer_algorithm, optimizer_objective, iterations=iterations, population=population)
print("\nBest objective (1 - F1):", best_objective)
print("Best hyperparameters:", best_hyper_params)



Running optimizer: PSO (pop=12, iters=12)


In [ ]:
print("\nRetraining best model on full training set and evaluating on test...")
model, test_metrics = retrain_and_evaluate(best_hyper_params, X_train.to_numpy(), y_train.to_numpy().ravel(), X_test.to_numpy(), y_test.to_numpy().ravel())
print("\nTest metrics @ threshold=0.5")
for k, v in test_metrics.items():
    print(f"  {k}: {v:.4f}")

In [ ]:
# TODO: export the best model